In [89]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as Func

import matplotlib.pyplot as plt

from tqdm import tqdm

In [90]:
torch.cuda.is_available()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [91]:
path='./Iris.csv'
fullData=pd.read_csv(path)
fullData=fullData.sample(frac=1)
fullData[0:4]

# dataloader=torch.utils.data.DataLoader(dataset, batch_size=16,shuffle=True, num_workers = 4)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
132,133,6.4,2.8,5.6,2.2,Iris-virginica
69,70,5.6,2.5,3.9,1.1,Iris-versicolor
146,147,6.3,2.5,5.0,1.9,Iris-virginica
56,57,6.3,3.3,4.7,1.6,Iris-versicolor


In [92]:
fullData=fullData.drop(columns='Id')
clases=fullData.iloc[:,4].unique()
print(clases)

['Iris-virginica' 'Iris-versicolor' 'Iris-setosa']


In [93]:
newLabel={
    'Iris-setosa':0,
    'Iris-versicolor':1,
    'Iris-virginica':2
}
fullData.replace(newLabel,inplace=True)

In [94]:
fullData

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
132,6.4,2.8,5.6,2.2,2
69,5.6,2.5,3.9,1.1,1
146,6.3,2.5,5.0,1.9,2
56,6.3,3.3,4.7,1.6,1
1,4.9,3.0,1.4,0.2,0
...,...,...,...,...,...
40,5.0,3.5,1.3,0.3,0
64,5.6,2.9,3.6,1.3,1
4,5.0,3.6,1.4,0.2,0
39,5.1,3.4,1.5,0.2,0


In [95]:

CountOfRows=fullData.shape[0]
testData=fullData[0:CountOfRows//3]
trainData=fullData[CountOfRows//3:CountOfRows]



In [96]:
def normalizeCollumns(in_file):
    for i in range(in_file.shape[1]):
        column=in_file[in_file.columns[i]]
        in_file[in_file.columns[i]]=(column-np.min(column))/(np.max(column)-np.min(column))
    return in_file

In [97]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,csv_file):
        data=csv_file
        x=data.iloc[:,:4]
        x=normalizeCollumns(x)
        y=data.iloc[:,4]
        self.x_train=torch.Tensor(np.array(x))
        self.y_train=nn.functional.one_hot(torch.Tensor(np.array(y)).to(torch.int64),num_classes=3)
        
        
    def __len__(self):
        return len(self.y_train)
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]

In [107]:
trainDataset=MyDataset(trainData)
trainDataloader= torch.utils.data.DataLoader(trainDataset,batch_size=5)

(tensor([0.1429, 0.1364, 0.5932, 0.6667]), tensor([0, 0, 1]))